# Tunning XGBoost, Catboost and Lightgbm Classifiers Hyperparameters with Hyperopt

## Import

### Modules

In [328]:
import warnings

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from hyperopt import STATUS_FAIL, STATUS_OK, Trials, fmin, hp, tpe
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, log_loss, roc_auc_score, get_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier


### Data

In [243]:
data = load_breast_cancer()
X = data['data']
y = data['target']

## Data Preprocessing

In [244]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.75)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# sss = StratifiedShuffleSplit(n_splits=1, train_size=0.9)
# for train_index, val_index in sss.split(X, y):
#     X_train, X_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

## Setup Estimators

Some basic parameters:

* `learning_rate` [X/L/C]: learning rate (alias: `eta` )
* `max_depth` [X/L/C]: maximum depth of trees
* `n_estimators` [X/L/C]: no. of boosting iterations
* `min_child_weight` [X/L]: minimum sum of instance (hessian) weight needed in a child
* `min_child_samples` [L/C]: minimum no. of data in one leaf
* `subsample` [X/L/C]: subsample ratio of the training instances (note that for CatBoost this parameter can be used only if either Poisson or Bernoulli bootstrap_type is * `selected`)
* `colsample_bytree` [X/L]: subsample ratio of columns in tree building
* `colsample_bylevel` [X/C]: subsample ratio of columns for each level in tree building
* `colsample_bynode` [X]: subsample ratio of columns for each node
* `tree_method` [X]: tree construction method
* `boosting` [L]: tree construction method
* `boosting_type` [C]: Ordered for ordered boosting or Plain for classic
* `early_stopping_rounds` [X/L/C]: parameter for fit() — stop the training if one metric of a validation data does not improve in last early_stopping_rounds rounds
* `eval_metric` [X/L/C]: evaluation metrics for validation data

In [181]:
hyperparameters = {
    "xgb_classifier": {
        'max_depth' : hp.randint('max_depth', 10),
        'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
        'n_estimators' : hp.randint('n_estimators', 250),
        'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
        'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)
        }

    ,"lgb_classifier": {
        'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
        'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
        'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
        'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
        'subsample':        hp.uniform('subsample', 0.8, 1),
        'n_estimators':     100,
    }

    ,"ctb_classifier": {
        'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
        'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
        'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
        'n_estimators':      100,
    }
}

In [182]:
class select_estimator:
    def __init__(self, X_train, X_val, y_train, y_val):
        self.X = {"train": X_train, "val": X_val}
        self.y = {"train": y_train, "val": y_val}
        self.loss = {"f1_score": []
                    ,"log_loss": []
                    ,"roc_auc_score": []
                    }
        self.metrics = {"f1_score": f1_score
                        ,"log_loss": log_loss
                        ,"roc_auc_score": roc_auc_score
                    }

    def xgb_classifier(self, parameters):
        estimator = XGBClassifier(**parameters, eval_metric="logloss")
        estimator.fit(self.X["train"], self.y["train"])
        return self.get_loss(estimator)

    def ctb_classifier(self, parameters):
        estimator = CatBoostClassifier(**parameters)
        estimator.fit(self.X["train"], self.y["train"])
        return self.get_loss(estimator)

    def lgb_classifier(self, parameters):
        estimator = LGBMClassifier(**parameters)
        estimator.fit(self.X["train"], self.y["train"])
        return self.get_loss(estimator)

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
            
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}

        return result, trials


    def get_loss(self, estimator):

        for dataset_type, X in self.X.items():
            y_pred_class = estimator.predict(X)
            y_pred_proba = estimator.predict_proba(X)[:, 1]

            for metric in self.loss.keys():
                if metric in ["f1_score"]:
                    self.loss[metric].append((1.0 - self.metrics[metric](self.y[dataset_type], y_pred_class))**2)

                elif metric in ["roc_auc_score"]:
                    self.loss[metric].append((1.0 - self.metrics[metric](self.y[dataset_type], y_pred_proba))**2)

                elif metric in ["log_loss"]:
                    self.loss[metric].append((self.metrics[metric](self.y[dataset_type], y_pred_class))**2)
                
        loss_df = pd.DataFrame.from_dict(self.loss)
        loss = loss_df.sum(axis=0).sum()

        return {'loss': np.sqrt(loss), 'status': STATUS_OK}

In [185]:
obj = select_estimator(X_train, X_val, y_train, y_val)

parameters = obj.process(fn_name='xgb_classifier', space=hyperparameters['xgb_classifier'], trials=Trials(), algo=tpe.suggest, max_evals=100)

# Simple XGBoost

In [224]:
space = {
        'max_depth' : hp.randint('max_depth', 10),
        'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
        'n_estimators' : hp.randint('n_estimators', 250),
        'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
        'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)
        }

In [225]:
def xgb_cost(space, X_train, y_train, X_val, y_val):

    warnings.filterwarnings(action='ignore')

    classifier = XGBClassifier(n_estimators = space['n_estimators']
                            ,max_depth = int(space['max_depth'])
                            ,learning_rate = space['learning_rate']
                            ,gamma = space['gamma']
                            ,min_child_weight = space['min_child_weight']
                            ,subsample = space['subsample']
                            ,colsample_bytree = space['colsample_bytree']
                            ,use_label_encoder=False
                            ,eval_metric="logloss"
                            )
    classifier.fit(X_train, y_train)
    y_proba_val = classifier.predict_proba(X_val)[:, 1]
    y_class_val = classifier.predict(X_val)

    rocauc_val = roc_auc_score(y_val, y_proba_val)
    f1_val = f1_score(y_val, y_class_val)
    logloss_val = log_loss(y_val, y_class_val)

    y_proba_train = classifier.predict_proba(X_train)[:, 1]
    y_class_train = classifier.predict(X_train)

    # Applying k-Fold Cross Validation
    rocauc_train = roc_auc_score(y_train, y_proba_train)
    f1_train = f1_score(y_train, y_class_train)
    logloss_train = log_loss(y_train, y_class_train)
    
    # print(f"CrossValMean: {CrossValMean}")
    train_proportions = X_train.shape[0]/(X_train.shape[0] + X_val.shape[0])

    loss = ((1.0 - rocauc_val)**2 + (1.0 - f1_val)**2 + logloss_val**2)/train_proportions
    loss += ((1.0 - rocauc_train)**2 + (1.0 - f1_train)**2 + logloss_train**2)/(1-train_proportions)

    return {'loss': np.sqrt(loss), 'status': STATUS_OK }

In [226]:
objective = lambda x: cost(x, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

In [227]:
trials = Trials()

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print("Best: ", best)

 # Fitting XGBoost to the Training set



100%|██████████| 100/100 [00:10<00:00,  9.55trial/s, best loss: 3.3280711111346004e-15]
Best:  {'colsample_bytree': 0.38, 'gamma': 0.22, 'learning_rate': 0.45, 'max_depth': 9, 'min_child_weight': 1.0, 'n_estimators': 179, 'subsample': 0.65}


In [228]:
best

{'colsample_bytree': 0.38,
 'gamma': 0.22,
 'learning_rate': 0.45,
 'max_depth': 9,
 'min_child_weight': 1.0,
 'n_estimators': 179,
 'subsample': 0.65}

In [125]:
classifier = XGBClassifier(**best)
classifier.fit(X_train, y_train)

 

# Applying k-Fold Cross Validation

from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

CrossValMean = accuracies.mean()

print("Final CrossValMean: ", CrossValMean)

 

CrossValSTD = accuracies.std()

 

# Predicting the Test set results

y_pred = classifier.predict(X_test)

y_pred = pd.DataFrame(y_pred)

y_pred.columns = ['Survived']

submission = submission.join(y_pred)

 

# Exporting dataset to csv

submission.to_csv("Titanic_Submission.csv", index=False, sep=',')

[14:05:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

NameError: name 'pd' is not defined

# Bayes Search CV

In [635]:
import sys
import traceback
from abc import ABC, abstractmethod
from collections.abc import Iterable
from typing import Union

from hyperopt import STATUS_FAIL, STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import KFold
from icecream import ic

class BayesSearchCV:

    def __init__(self, estimator, param_distributions: dict, scoring: dict
        ,n_iter: int=10, cv: Union[int, Iterable]=5, random_state: int=None
        ,algo=tpe.suggest, trials: Trials=Trials()) -> None:
        """[summary]

        Args:
            estimator: Estimator object
            param_distributions (dict): Search space containing hyperparameters
            scoring (dict): {metric: opt_value} Dict of performance metrics to measure the estimator performance and their corresponding optimal values.
                                Select one from sklearn.metrics.SCORERS.keys()
            n_iter (int, optional): Max number of iterations. Defaults to 10.
            cv (int or Iterable, optional): int, cross-validation generator or an iterable. Defaults to 5.
            random_state (int, optional): Pseudo random number generator state used for random uniform sampling. Defaults to None.
            algo (optional): Algorithm to for distribution search. Defaults to tpe.suggest.
            trials (Trials, optional): [description]. Defaults to Trials().
        """
        self.estimator = estimator
        self.param_distributions = param_distributions
        self.n_iter = n_iter
        self.random_state = random_state
        self.cv = cv
        self.algo = algo
        self.trials = trials
        self.scoring = scoring

    def fit(self, X: pd.DataFrame, y=None):
        """Fit estimator with optimal hyperparameters

        Args:
            X (pd.DataFrame): Predictors
            y (pd.DataFrame): Target
        """
        self.cv_results_ = pd.DataFrame()
        self.min_loss = np.inf

        for train_index, val_index in self._get_splits(X, y):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            objective = lambda space: self._cost(X=X_train, y=y_train, hyperparameters=space)

            try:
                hyperparameters = fmin(fn=objective, space=self.param_distributions
                            ,algo=self.algo, max_evals=self.n_iter
                            ,trials=self.trials)

            except KeyError:
                exc_info = sys.exc_info()
                traceback.print_exception(*exc_info)
                return {'status': STATUS_FAIL,
                        'exception': str(sys.exc_info())}

            estimator, loss_df, current_loss = self._cost(X_val, y_val, hyperparameters, return_fit_estimator=True)

            if current_loss < self.min_loss:
                self.min_loss = current_loss
                self.best_estimator_ = estimator
                self.best_hyperparameters_ = hyperparameters

            self.cv_results_ = pd.concat([self.cv_results_ , loss_df.copy()])

        self.cv_results_.reset_index(inplace=True, drop=True)
        self.cv_results_.rename(columns={col: f"{col}_loss" for col in self.cv_results_.columns if "loss" not in col}, inplace=True)


    def _get_splits(self, X: pd.DataFrame, y=None):
        """Instantiate and/or get training and validation datasets

        Args:
            X (pd.DataFrame): Predictor
            y (pd.DataFrame): Target

        Yields:
            [type]: Train and test indices

        Raises:
            NotImplementedError: Only KFold and StratifiedKFold are implemented
        """

        if isinstance(self.cv, int): 
            self.cv = KFold(n_splits=self.cv, random_state=self.random_state)

        elif isinstance(self.cv, StratifiedKFold):
            pass

        else:
            msg = f"Cross validation not yet implemented for type {type(self.cv)}"
            NotImplementedError(msg)
            
        for train_index, test_index in self.cv.split(X, y):
            yield train_index, test_index


    def _cost(self, X: pd.DataFrame, y: pd.DataFrame, hyperparameters: dict
            ,return_fit_estimator: bool=False, fit_estimator=None) -> dict:
        """Evaluates the cost function for the trained estimator

        Args:
            X (pd.DataFrame): Predictor
            y (pd.DataFrame): Target
            hyperarameters (dict): Estimator hyperparameters
            return_fit_estimator (bool): Returns fit estimator
            fit_estimator: Fit estimator

        Returns:
            (dict)
        """
        loss_dict = {metric_name: [] for metric_name in self.scoring}

        if not fit_estimator:
            estimator = self._instantiate_estimator(hyperparameters)
            estimator.fit(X, y)

        for p_metric, opt_value in self.scoring.items():
            scorer = get_scorer(p_metric)
            loss_dict[p_metric].append((opt_value - scorer(estimator, X, y))**2)

        loss_df = pd.DataFrame.from_dict(loss_dict)
        loss = loss_df.sum(axis=0).sum()
        loss_df["loss"] = loss_df.sum(axis=1)

        if return_fit_estimator:
            return estimator, loss_df, loss

        return {'loss': np.sqrt(loss), 'status': STATUS_OK}


    def _instantiate_estimator(self, hyperarameters: dict):
        """Instantiate estimator with selected hyperparameters

        Args:
            hyperarameters (dict):

        Returns:
            [type]: Estimator
        """
        estimator_cls = self.estimator.__class__
        return estimator_cls(**hyperarameters)


In [629]:
space = {
        'max_depth' : hp.randint('max_depth', 10),
        'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
        'n_estimators' : hp.randint('n_estimators', 250),
        'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
        'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01),
        'eval_metric': "logloss"
        }

cv = BayesSearchCV(XGBClassifier(learning_rate=0.1), param_distributions=space, scoring={"roc_auc": 1, "f1": 1}, cv=StratifiedKFold())
cv.fit(X_train, y_train)
print(cv.cv_results_)
print(cv.best_hyperparameters_)
print(cv.min_loss)

100%|██████████| 10/10 [00:00<00:00, 13.41trial/s, best loss: 0.004672897196261627]
[19:53:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[19:53:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[19:53:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 10/10 [00:00<?, ?trial/s, best 

In [631]:
space = {
        'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
        'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
        'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
        'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
        'subsample':        hp.uniform('subsample', 0.8, 1),
        # 'n_estimators':     100,
    }
cv = BayesSearchCV(LGBMClassifier(), param_distributions=space, scoring={"roc_auc": 1, "f1": 1}, cv=StratifiedKFold())
cv.fit(X_train, y_train)
print(cv.cv_results_)
print(cv.best_hyperparameters_)
print(cv.min_loss)

100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[LightGBM] [Warning] Unknown parameter: gamma
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[LightGBM] [Warning] Unknown parameter: gamma
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[LightGBM] [Warning] Unknown parameter: gamma
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[LightGBM] [Warning] Unknown parameter: gamma
100%|██████████| 10/10 [00:00<?, ?trial/s, best loss=?]
[LightGBM] [Warning] Unknown parameter: gamma
   roc_auc_loss   f1_loss      loss
0      0.000001  0.000331  0.000332
1      0.000006  0.000730  0.000736
2      0.000000  0.000087  0.000087
3      0.000000  0.000091  0.000091
4      0.000000  0.000000  0.000000
{'colsample_bytree': 0.44, 'gamma': 0.4, 'learning_rate': 0.13, 'max_depth': 1, 'min_child_weight': 3.0, 'n_estimators': 132, 'subsample': 0.9500000000000001}
0.0


In [636]:
space = {
        'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
        'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
        'n_estimators':      100,
    }
cv = BayesSearchCV(CatBoostClassifier(), param_distributions=space, scoring={"roc_auc": 1, "f1": 1}, cv=StratifiedKFold())
cv.fit(X_train, y_train)
print(cv.cv_results_)
print(cv.best_hyperparameters_)
print(cv.min_loss)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.4880867	total: 1.02s	remaining: 1m 41s
1:	learn: 0.3478351	total: 1.19s	remaining: 58.3s
2:	learn: 0.2704963	total: 1.38s	remaining: 44.6s
3:	learn: 0.2173276	total: 1.55s	remaining: 37.3s
4:	learn: 0.1578147	total: 1.72s	remaining: 32.8s
5:	learn: 0.1318153	total: 1.86s	remaining: 29.2s
6:	learn: 0.1151857	total: 2.02s	remaining: 26.8s
7:	learn: 0.0974524	total: 2.16s	remaining: 24.8s
8:	learn: 0.0862268	total: 2.3s	remaining: 23.2s
9:	learn: 0.0766485	total: 2.44s	remaining: 21.9s
10:	learn: 0.0685282	total: 2.58s	remaining: 20.8s
11:	learn: 0.0621663	total: 2.7s	remaining: 19.8s
12:	learn: 0.0549267	total: 2.84s	remaining: 19s
13:	learn: 0.0502857	total: 2.99s	remaining: 18.4s
14:	learn: 0.0469769	total: 3.13s	remaining: 17.7s
15:	learn: 0.0435388	total: 3.25s	remaining: 17s
16:	learn: 0.0400515	total: 3.39s	remaining: 16.5s
17:	learn: 0.0362854	total: 3.58s	remaining: 16.3s
18:	learn: 0.0341735	total: 3.77s	remaining: 16.1s
19:	learn: 0.0320816	total: 3.96s	remaining: 1

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


11:	learn: 0.0014252	total: 80.4ms	remaining: 6.62s
12:	learn: 0.0013388	total: 87.8ms	remaining: 6.67s
13:	learn: 0.0012633	total: 91.2ms	remaining: 6.42s
14:	learn: 0.0011969	total: 94.7ms	remaining: 6.22s
15:	learn: 0.0010985	total: 98.7ms	remaining: 6.07s
16:	learn: 0.0010306	total: 103ms	remaining: 5.97s
17:	learn: 0.0010290	total: 105ms	remaining: 5.75s
18:	learn: 0.0009710	total: 108ms	remaining: 5.58s
19:	learn: 0.0009477	total: 110ms	remaining: 5.41s
20:	learn: 0.0009453	total: 121ms	remaining: 5.65s
21:	learn: 0.0009047	total: 125ms	remaining: 5.57s
22:	learn: 0.0008916	total: 129ms	remaining: 5.48s
23:	learn: 0.0008894	total: 132ms	remaining: 5.38s
24:	learn: 0.0008745	total: 136ms	remaining: 5.29s
25:	learn: 0.0008593	total: 140ms	remaining: 5.24s
26:	learn: 0.0008473	total: 144ms	remaining: 5.19s
27:	learn: 0.0008423	total: 148ms	remaining: 5.14s
28:	learn: 0.0008417	total: 151ms	remaining: 5.05s
29:	learn: 0.0008408	total: 154ms	remaining: 4.99s
30:	learn: 0.0008399	total

...

17:	learn: 0.0009765	total: 70ms	remaining: 3.82s
18:	learn: 0.0009172	total: 76.6ms	remaining: 3.95s
19:	learn: 0.0008790	total: 82.5ms	remaining: 4.04s
20:	learn: 0.0008315	total: 88.2ms	remaining: 4.11s
21:	learn: 0.0008282	total: 97.8ms	remaining: 4.35s
22:	learn: 0.0007934	total: 101ms	remaining: 4.28s
23:	learn: 0.0007618	total: 105ms	remaining: 4.25s
24:	learn: 0.0007329	total: 108ms	remaining: 4.2s
25:	learn: 0.0007152	total: 110ms	remaining: 4.11s
26:	learn: 0.0007126	total: 112ms	remaining: 4.04s
27:	learn: 0.0007056	total: 115ms	remaining: 3.98s
28:	learn: 0.0006788	total: 117ms	remaining: 3.92s
29:	learn: 0.0006562	total: 119ms	remaining: 3.86s
30:	learn: 0.0006373	total: 122ms	remaining: 3.82s
31:	learn: 0.0006358	total: 126ms	remaining: 3.81s
32:	learn: 0.0006219	total: 129ms	remaining: 3.77s
33:	learn: 0.0006122	total: 132ms	remaining: 3.74s
34:	learn: 0.0005941	total: 135ms	remaining: 3.73s
35:	learn: 0.0005937	total: 138ms	remaining: 3.69s
36:	learn: 0.0005887	total: 1

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


16:	learn: 0.0009926	total: 70.4ms	remaining: 4.07s
17:	learn: 0.0009475	total: 78.9ms	remaining: 4.3s
18:	learn: 0.0009067	total: 81.3ms	remaining: 4.2s
19:	learn: 0.0008696	total: 85.1ms	remaining: 4.17s
20:	learn: 0.0008661	total: 89.2ms	remaining: 4.16s
21:	learn: 0.0008627	total: 92.4ms	remaining: 4.11s
22:	learn: 0.0008343	total: 94.8ms	remaining: 4.02s
23:	learn: 0.0008093	total: 97.2ms	remaining: 3.95s
24:	learn: 0.0007803	total: 99.4ms	remaining: 3.88s
25:	learn: 0.0007694	total: 102ms	remaining: 3.81s
26:	learn: 0.0007434	total: 104ms	remaining: 3.75s
27:	learn: 0.0007336	total: 108ms	remaining: 3.74s
28:	learn: 0.0007164	total: 112ms	remaining: 3.74s
29:	learn: 0.0007002	total: 115ms	remaining: 3.72s
30:	learn: 0.0006980	total: 118ms	remaining: 3.68s
31:	learn: 0.0006958	total: 127ms	remaining: 3.83s
32:	learn: 0.0006937	total: 129ms	remaining: 3.79s
33:	learn: 0.0006916	total: 133ms	remaining: 3.77s
34:	learn: 0.0006895	total: 135ms	remaining: 3.73s
35:	learn: 0.0006752	tot

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


15:	learn: 0.0010400	total: 80.9ms	remaining: 4.98s
16:	learn: 0.0009792	total: 84ms	remaining: 4.86s
17:	learn: 0.0009783	total: 88.5ms	remaining: 4.83s
18:	learn: 0.0009773	total: 92.3ms	remaining: 4.76s
19:	learn: 0.0009771	total: 95.3ms	remaining: 4.67s
20:	learn: 0.0009658	total: 98.6ms	remaining: 4.59s
21:	learn: 0.0009655	total: 102ms	remaining: 4.53s
22:	learn: 0.0009144	total: 105ms	remaining: 4.45s
23:	learn: 0.0009024	total: 110ms	remaining: 4.46s
24:	learn: 0.0009009	total: 113ms	remaining: 4.41s
25:	learn: 0.0008993	total: 116ms	remaining: 4.34s
26:	learn: 0.0008409	total: 120ms	remaining: 4.31s
27:	learn: 0.0008403	total: 123ms	remaining: 4.27s
28:	learn: 0.0008243	total: 126ms	remaining: 4.22s
29:	learn: 0.0008181	total: 129ms	remaining: 4.18s
30:	learn: 0.0008145	total: 134ms	remaining: 4.17s
31:	learn: 0.0008106	total: 136ms	remaining: 4.13s
32:	learn: 0.0008094	total: 140ms	remaining: 4.12s
33:	learn: 0.0008091	total: 143ms	remaining: 4.08s
34:	learn: 0.0008085	total:

In [517]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [453]:
get_scorer("f1_score")

ValueError: 'f1_score' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [296]:
module = __import__()
class_ = getattr(module, model.__class__.__name__)
instance = class_()

ModuleNotFoundError: No module named 'XGBClassifier'

In [379]:
a = XGBClassifier(learning_rate=0.1, eval_metric="logloss")